In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh
from scipy.sparse.linalg import eigs
import scipy.io
import sys
sys.path.append("../")
from algorithm.KoopPseudoSpec import KoopPseudoSpec
from algorithm.PhasePlot import PhasePlot
# import mat73

In [2]:
# Load data from above computation - file available from the dropbox link

data = scipy.io.loadmat('pendulum_data.mat')

In [6]:
A = data['A']
N1 = int(data['N1'])
N2 = int(data['N2'])

# Create index arrays
Id1 = np.maximum(np.kron(np.abs(np.arange(-N1, N1+1)), np.ones(N2)), 1)
Id2 = np.kron(np.ones(2*N1+1), np.arange(1, N2+1))

# Apply condition and reshape A
N_trun = N2
Id = np.where((np.abs(Id1) * np.abs(Id2)) < (N_trun + 1))[0]
A = A[Id, :][:, Id]

# Compute pseudospectra
x_pts = np.arange(-1.5, 1.5, 0.05)
y_pts = x_pts
z_pts = np.outer(x_pts, np.ones(len(y_pts))) + 1j * np.outer(np.ones(len(x_pts)), y_pts)
z_pts = z_pts.flatten()

# Assuming KoopPseudoSpec is a function you have in Python
RES = KoopPseudoSpec(np.eye(len(A)), A, np.eye(len(A)), z_pts)
RES = RES.reshape(len(y_pts), len(x_pts))

# EDMD eigenvalues
E = eigh(A)[0]

# Plotting
plt.figure()
# Assuming you have the RES computed
# plt.contourf(z_pts.real.reshape(len(y_pts), len(x_pts)), z_pts.imag.reshape(len(y_pts), len(x_pts)), np.log10(1./RES.real), levels=np.log10([1e-50, 0.25]), colors='k', linewidths=1.5)
plt.gca().set_aspect('equal', 'box')
plt.plot(E.real, E.imag, '.m')
plt.plot(np.exp(1j * np.linspace(0, 2*np.pi, 1000)).real, np.exp(1j * np.linspace(0, 2*np.pi, 1000)).imag, '-r')

# Compute approximate eigenvalues and eigenfunctions
E_test = [0.880347701848197 + 0.473226384533343j,
          0.558620606228836 + 0.826563250517901j,
          0.124674806328568 + 0.987209196182015j,
          -0.314745042094750 + 0.936136228237800j]
z = E_test[3]
z = z / abs(z)

xp1 = np.linspace(-np.pi, np.pi, 500)
xp1 = xp1 + (xp1[1] - xp1[0]) / 2
xp1 = xp1[:-1]
xp2 = np.linspace(-4, 4, len(xp1))
XpH = np.zeros((len(xp1), N2))
XpH[:, 0] = np.exp(-0.5 * xp2**2) / np.pi**0.25
XpH[:, 1] = XpH[:, 0] * np.sqrt(2) * xp2
for j in range(2, N2):
    XpH[:, j] = np.sqrt(2) / np.sqrt(j-1) * XpH[:, j-1] * xp2 - np.sqrt(j-2) / np.sqrt(j-1) * XpH[:, j-2]
XpF = np.exp(1j * np.outer(np.arange(-N1, N1+1), xp1))

H = np.eye(len(A)) - (z * A.T) - (z * A.T).conj() + abs(z)**2 * np.eye(len(A))
V, D = eigs(H, k=1, which='SM')
V2 = np.zeros(N2 * (2 * N1 + 1))
V2[Id] = V
V = V2.reshape(N2, 2*N1+1)
PSI = (XpF @ V.T @ XpH.T).T

# Assuming PhasePlot is a function you have in Python
PhasePlot(xp1 + 1j * xp2, PSI, 'm')
plt.gca().fontsize = 14
plt.show()

[[-2.24111143e-16-1.48613804e-16j -8.39620911e-17-1.22020490e-15j
  -1.81262532e-16+8.01817510e-17j ... -8.86967707e-18+1.44827380e-15j
   8.44599535e-16-2.97651241e-17j -2.38812908e-17+8.87773462e-16j]
 [-4.73655608e-17-1.02012011e-15j  3.35661852e-16-1.45648716e-16j
   1.11173989e-16+9.42093652e-16j ... -1.10795278e-15-4.15585500e-17j
  -2.50640508e-17+4.16911101e-16j  6.19790848e-16+1.00777755e-16j]
 [ 6.75076124e-16-6.23029108e-17j  2.00874741e-17-1.23057407e-15j
   1.87912031e-16-1.54932359e-16j ...  6.74516702e-17+7.36125764e-16j
  -6.59645313e-16+7.15370827e-18j  2.90049165e-18+1.10987249e-15j]
 ...
 [ 1.40848529e-17-4.85716577e-16j -8.84277837e-16+5.22775407e-17j
   6.83995841e-17-8.24840049e-16j ...  5.94007588e-16+2.03813672e-16j
  -6.65642589e-17+9.77964718e-16j -1.55450385e-16-4.59848021e-17j]
 [ 8.44599535e-16+2.97651241e-17j -2.38812908e-17-8.87773462e-16j
  -3.23475215e-16-2.40020688e-18j ...  6.43698978e-17-9.60204211e-16j
  -2.24111143e-16+1.48613804e-16j -8.39620911e-

ArpackNoConvergence: ARPACK error -1: No convergence (9641 iterations, 0/1 eigenvectors converged) [ARPACK error -14: ZNAUPD did not find any eigenvalues to sufficient accuracy.]